In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,el calafate,-50.3408,-72.2768,5.33,77,3,2.87,AR,1696295705
1,1,grand falls-windsor,48.9332,-55.6649,10.28,74,23,1.91,CA,1696295705
2,2,bubaque,11.2833,-15.8333,28.05,81,99,2.78,GW,1696295705
3,3,baruun-urt,46.6806,113.2792,7.05,45,35,4.99,MN,1696295705
4,4,fortuna,40.5982,-124.1573,16.19,79,0,5.66,US,1696295559


In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",  # Use the 'Humidity' column for point size
    scale=1,
    color="Humidity",  # Use the 'Humidity' column for point color
    cmap="coolwarm",  # Specify a colormap
    hover_cols=["City", "Humidity"]  # Display city and humidity in hover
)

# Display the humidity map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5)&
     (city_data_df['Cloudiness']==0)
]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
82,82,colonia,50.9333,6.9500,21.90,77,0,2.06,DE,1696295599
90,90,hamilton,39.1834,-84.5333,23.55,73,0,0.89,US,1696295709
153,153,zaragoza,41.6561,-0.8773,21.68,73,0,3.09,ES,1696295643
187,187,laguna,38.4210,-121.4238,26.17,30,0,0.45,US,1696295713
195,195,superior,46.7208,-92.1041,24.26,81,0,2.57,US,1696295714
227,227,monroeville,40.4212,-79.7881,21.94,76,0,1.01,US,1696295506
259,259,karkira,36.9292,6.5856,21.37,69,0,2.11,DZ,1696295717
267,267,roccella ionica,38.3220,16.4009,21.27,59,0,2.15,IT,1696295718
285,285,al ghayzah,16.2079,52.1760,25.20,81,0,1.66,YE,1696295719
301,301,al muwayh,22.4333,41.7583,24.27,22,0,2.69,SA,1696295720


In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
82,colonia,DE,50.9333,6.9500,77,
90,hamilton,US,39.1834,-84.5333,73,
153,zaragoza,ES,41.6561,-0.8773,73,
187,laguna,US,38.4210,-121.4238,30,
195,superior,US,46.7208,-92.1041,81,
227,monroeville,US,40.4212,-79.7881,76,
259,karkira,DZ,36.9292,6.5856,69,
267,roccella ionica,IT,38.3220,16.4009,59,
285,al ghayzah,YE,16.2079,52.1760,81,
301,al muwayh,SA,22.4333,41.7583,22,


In [6]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    radius = 10000

    limit = 20
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    

    params = {
    "categories" : "accommodation.hotel",
    "limit": limit,
    "filter" : filters,
    "bias" : bias ,
    "apiKey" : geoapify_key
}

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make an API request using the params dictionaty and jsonify it
    name_address = requests.get(base_url, params=params)
    
    name_address_json = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
colonia - nearest hotel: Wasserturm Hotel Cologne
hamilton - nearest hotel: North Vista Manor
zaragoza - nearest hotel: Hotel Pilar Plaza
laguna - nearest hotel: Holiday Inn Express & Suites
superior - nearest hotel: Androy Hotel
monroeville - nearest hotel: DoubleTree by Hilton Hotel Pittsburgh - Monroeville Convention Center
karkira - nearest hotel: استراحة دريم هاوس
roccella ionica - nearest hotel: Mediterraneo
al ghayzah - nearest hotel: فندق تاج العرب
al muwayh - nearest hotel: فندق ومطعم الموية
fort bragg - nearest hotel: Airborne Inn Lodging
'alaqahdari dishu - nearest hotel: No hotel found
arifwala - nearest hotel: Noor mahal banquet hall
talagang - nearest hotel: Al husnain hotel
kolonia - nearest hotel: Wasserturm Hotel Cologne
jati - nearest hotel: No hotel found
carbondale - nearest hotel: Quality Inn Carbondale
as sulayyil - nearest hotel: No hotel found
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
fredericksburg - nearest hotel: Kenmore Inn
brooki

,City,Country,Lat,Lng,Humidity,Hotel Name
82,colonia,DE,50.9333,6.9500,77,Wasserturm Hotel Cologne
90,hamilton,US,39.1834,-84.5333,73,North Vista Manor
153,zaragoza,ES,41.6561,-0.8773,73,Hotel Pilar Plaza
187,laguna,US,38.4210,-121.4238,30,Holiday Inn Express & Suites
195,superior,US,46.7208,-92.1041,81,Androy Hotel
227,monroeville,US,40.4212,-79.7881,76,DoubleTree by Hilton Hotel Pittsburgh - Monroe...
259,karkira,DZ,36.9292,6.5856,69,استراحة دريم هاوس
267,roccella ionica,IT,38.3220,16.4009,59,Mediterraneo
285,al ghayzah,YE,16.2079,52.1760,81,فندق تاج العرب
301,al muwayh,SA,22.4333,41.7583,22,فندق ومطعم الموية


In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",  # Use the 'Humidity' column for point size
    scale=1,
    color="Humidity",  # Use the 'Humidity' column for point color
    cmap="coolwarm",  # Specify a colormap
    hover_cols=["City", "Country", "Hotel Name"]  # Display city, Country, Hotel Name in hover
)

# Display the hotel map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)